<a href="https://colab.research.google.com/github/mbjallow6/Algorithms-python/blob/main/Rossalind_Problems_Part_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# import data from the computer
from google.colab import files
uploaded = files.upload()

Saving rosalind_rnas.txt to rosalind_rnas.txt


## Counting Valid RNA Secondary Structures with Wobble Pairs

In [ ]:
"""
Rosalind Counting Valid RNA Secondary Structures with Wobble Pairs

This module uses iterative dynamic programming to count valid noncrossing matchings
in RNA bonding graphs, allowing wobble pairs and minimum distance constraints.

Author: Bioinformatics Solution
Compatible with: Python 3.6+
Platform: Google Colab
"""

from typing import List, Tuple

class RNAMatchingCounter:
    """
    Class to count valid RNA matchings.
    """

    def __init__(self):
        """Initialize base pair rules."""
        self.base_pairs = {
            ('A', 'U'), ('U', 'A'),
            ('C', 'G'), ('G', 'C'),
            ('G', 'U'), ('U', 'G')
        }

    def can_pair(self, base1: str, base2: str) -> bool:
        """Check if two bases can pair."""
        return (base1, base2) in self.base_pairs

    def count_valid_matchings(self, rna: str) -> int:
        """
        Count valid matchings using 2D DP.

        dp[i][j]: number of valid matchings for subsequence rna[i..j]
        """
        n = len(rna)
        if n == 0:
            return 1

        dp: List[List[int]] = [[0] * n for _ in range(n)]

        # Base cases: single base or empty
        for i in range(n):
            dp[i][i] = 1  # Unpaired single base

        for length in range(1, n):
            for i in range(n - length):
                j = i + length
                # Case: j is unpaired
                dp[i][j] = dp[i][j - 1]
                # Case: j pairs with some k
                for k in range(i, j):
                    if self.can_pair(rna[k], rna[j]) and (j - k) >= 4:
                        left = dp[i][k - 1] if k > i else 1
                        inside = dp[k + 1][j - 1] if k + 1 <= j - 1 else 1
                        dp[i][j] += left * inside

        return dp[0][n - 1]

    def verify_count(self, rna: str, count: int) -> Tuple[bool, str]:
        """Verify against known small cases."""
        known = {
            "CGAUGCUAG": 12,
            "AU": 1,
            "AUGC": 1,
            "AUGCAU": 2,
            "AUGCUAGUACGGAGCGAGUCUAGCGAGCGAUGUCGUGAGUACUAUAUAUGCGCAUAAGCCACGU": 284850219977421
        }
        if rna in known:
            if count == known[rna]:
                return True, "Valid: Matches expected"
            return False, f"Invalid: Expected {known[rna]}, got {count}"
        return True, "No verification available"

def parse_input_file(file_path: str) -> str:
    """Parse RNA from file."""
    with open(file_path, 'r') as f:
        rna = ''.join(line.strip() for line in f if line.strip())
    if not all(b in 'AUGC' for b in rna):
        raise ValueError("Invalid RNA sequence")
    return rna

def write_output_file(output_path: str, result: int) -> None:
    """Write result to file."""
    with open(output_path, 'w') as f:
        f.write(str(result) + '\n')

def solve_problem(input_file: str, output_file: str) -> None:
    """Solve and output."""
    rna = parse_input_file('/content/rosalind_rnas.txt')
    counter = RNAMatchingCounter()
    result = counter.count_valid_matchings(rna)
    is_valid, msg = counter.verify_count(rna, result)
    if not is_valid:
        raise ValueError(msg)
    write_output_file(output_file, result)
    print(f"Result: {result} (Verification: {msg})")

# Usage example (adjust file names as needed)
if __name__ == "__main__":
    input_file = "rosalind_input.txt"
    output_file = "rosalind_output.txt"
    solve_problem(input_file, output_file)


## Rosalind Distances in Trees Problem

In [ ]:
import re

class Node:
    """A simple class to represent a node in a tree."""
    def __init__(self, name=None, parent=None):
        self.name = name
        self.parent = parent
        self.children = []

    def __repr__(self):
        return f"Node({self.name})"

def parse_newick(newick_string):
    """
    Parses a Newick format string into a tree of Node objects.

    Returns a dictionary mapping all named nodes to their Node objects.
    """
    # Clean up the string by removing the final semicolon and any whitespace
    tokens = re.split(r'([,();])', newick_string.strip())
    tokens = [t.strip() for t in tokens if t.strip()]

    root = Node()
    current_node = root
    nodes_map = {}

    # Counter for generating unique names for internal nodes that are not explicitly named
    internal_node_counter = 0

    for token in tokens:
        if token == '(':
            # Start of a new set of children. Create a new child node.
            # If the current node doesn't have a name yet, it's an internal node.
            if current_node.name is None:
                current_node.name = f"internal_{internal_node_counter}"
                internal_node_counter += 1
            if current_node.name not in nodes_map:
                nodes_map[current_node.name] = current_node

            # Create a new child and descend into it
            new_child = Node(parent=current_node)
            current_node.children.append(new_child)
            current_node = new_child

        elif token == ',':
            # A sibling follows. Go back to the parent to add the next sibling.
            current_node = current_node.parent
            new_sibling = Node(parent=current_node)
            current_node.children.append(new_sibling)
            current_node = new_sibling

        elif token == ')':
            # End of a children list. Go back to the parent.
            current_node = current_node.parent

        elif token == ';':
            # End of the entire tree string
            break

        else:
            # This token is a name for the current node.
            current_node.name = token
            nodes_map[token] = current_node

    # If the root node was a simple leaf (e.g., "dog;"), it won't be in the map yet.
    if root.name and root.name not in nodes_map:
        nodes_map[root.name] = root

    return nodes_map

def solve_paths_in_trees_no_bio():
    """
    Reads trees, calculates distances without Biopython, and writes results.
    """
    try:
        with open('rosalind_nwck.txt', 'r') as f:
            data_sets = f.read().strip().split('\n\n')

        distances = []

        for data_set in data_sets:
            if not data_set:
                continue

            newick_string, nodes_line = data_set.strip().split('\n')
            node1_name, node2_name = nodes_line.split()

            # Step 1: Parse the Newick string into a tree
            nodes_map = parse_newick(newick_string)

            # Get the node objects from the map
            node1 = nodes_map[node1_name]
            node2 = nodes_map[node2_name]

            # Step 2: Find the path from node1 to the root
            path_to_root1 = []
            curr = node1
            while curr:
                path_to_root1.append(curr)
                curr = curr.parent

            # Create a set for quick lookups of ancestors
            ancestors1 = set(path_to_root1)

            # Step 3: Find the LCA by traversing from node2 to the root
            lca = None
            path_to_lca2 = []
            curr = node2
            while curr:
                path_to_lca2.append(curr)
                if curr in ancestors1:
                    lca = curr
                    break
                curr = curr.parent

            # Step 4: Calculate the distance
            # The distance is the number of edges, which is (path_len1 - 1) + (path_len2 - 1)
            dist1 = path_to_root1.index(lca)
            dist2 = path_to_lca2.index(lca)
            total_distance = dist1 + dist2
            distances.append(str(total_distance))

        with open('output_nwck.txt', 'w') as f:
            f.write(' '.join(distances))

        print("Processing complete. Results are in 'output_nwck.txt'.")

    except FileNotFoundError:
        print("Error: 'rosalind_nwck.txt' not found.")
        print("Please create the file and add your input data.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Execute the function
solve_paths_in_trees_no_bio()



## Motzkin Numbers and RNA Secondary Structures

In [ ]:
"""
Rosalind Motzkin Numbers and RNA Secondary Structures Solution

This module implements a dynamic programming approach based on Motzkin numbers
to count noncrossing matchings in RNA secondary structures. It handles large inputs
efficiently and includes verification for correctness.

Author: Bioinformatics Solution
Compatible with: Python 3.6+
Platform: Google Colab
"""

from typing import Dict, Tuple

class MotzkinRNA:
    """
    A class to compute the number of noncrossing matchings in RNA secondary structures
    using a modified Motzkin recurrence.
    """

    def __init__(self):
        """Initialize the Motzkin RNA counter with base pair rules."""
        self.base_pairs = {('A', 'U'), ('U', 'A'), ('G', 'C'), ('C', 'G')}
        self.mod = 1000000

    def can_pair(self, base1: str, base2: str) -> bool:
        """
        Check if two RNA bases can form a pair.

        Args:
            base1 (str): First RNA base
            base2 (str): Second RNA base

        Returns:
            bool: True if bases can pair, False otherwise
        """
        return (base1, base2) in self.base_pairs

    def count_noncrossing_matchings(self, rna: str) -> int:
        """
        Count the number of noncrossing matchings in the RNA bonding graph using
        dynamic programming based on Motzkin numbers.

        Args:
            rna (str): RNA sequence

        Returns:
            int: Number of noncrossing matchings modulo 1,000,000
        """
        n = len(rna)
        if n == 0:
            return 1

        # dp[i][j] represents the number of noncrossing matchings for subsequence i to j
        dp = [[0] * n for _ in range(n)]

        # Initialize base cases for single and empty subsequences
        for i in range(n):
            dp[i][i] = 1  # Single base, no pairing
            if i + 1 < n:
                dp[i][i+1] = 1  # Two bases, no pairing yet

        # Fill the dp table for subsequences of length 2 to n
        for length in range(2, n + 1):
            for start in range(n - length + 1):
                end = start + length - 1
                # Case 1: Last base is unpaired
                dp[start][end] = dp[start][end - 1]

                # Case 2: Last base pairs with some base k
                for k in range(start, end):
                    if self.can_pair(rna[k], rna[end]):
                        if k == start:
                            # Pairing with first base, count inside
                            inside = dp[start + 1][end - 1] if start + 1 <= end - 1 else 1
                            dp[start][end] = (dp[start][end] + inside) % self.mod
                        else:
                            # Split into two parts: start to k-1 and k+1 to end-1
                            left = dp[start][k - 1] if start <= k - 1 else 1
                            right = dp[k + 1][end - 1] if k + 1 <= end - 1 else 1
                            dp[start][end] = (dp[start][end] + left * right) % self.mod

        return dp[0][n - 1]

    def verify_count(self, rna: str, count: int) -> Tuple[bool, str]:
        """
        Verify the count of noncrossing matchings for small test cases.

        Args:
            rna (str): RNA sequence
            count (int): Computed number of matchings

        Returns:
            Tuple[bool, str]: (is_valid, error_message)
        """
        # For small known cases, verify against expected output
        known_cases = {
            "AUAU": 7,
            "AU": 2,
            "A": 1,
            "": 1
        }
        if rna in known_cases:
            expected = known_cases[rna]
            if count == expected:
                return True, f"Valid: Matches expected count {expected} for {rna}"
            return False, f"Invalid: Expected {expected} for {rna}, got {count}"
        return True, "No verification for this input (unknown case)"

def parse_input_file(file_path: str) -> str:
    """
    Parse input file to extract RNA sequence.

    Args:
        file_path (str): Path to input file

    Returns:
        str: RNA sequence
    """
    try:
        with open(file_path, 'r') as file:
            lines = [line.strip() for line in file if line.strip()]

        if not lines:
            raise ValueError("Input file is empty")

        # Skip the header line if it starts with '>'
        rna = ''.join(lines[1:]) if lines[0].startswith('>') else ''.join(lines)
        if not all(base in 'AUGC' for base in rna):
            raise ValueError("Invalid RNA sequence: must contain only A, U, G, C")
        return rna

    except FileNotFoundError:
        raise FileNotFoundError(f"Input file '{file_path}' not found")
    except Exception as e:
        raise ValueError(f"Input parsing error: {e}")

def write_output_file(output_path: str, result: int) -> None:
    """
    Write the result to the output file.

    Args:
        output_path (str): Path to output file
        result (int): Number of noncrossing matchings
    """
    try:
        with open(output_path, 'w') as file:
            file.write(str(result) + '\n')
    except Exception as e:
        raise IOError(f"Error writing to output file: {e}")

def solve_motzkin_rna_problem(input_file_path: str) -> int:
    """
    Solve the Motzkin RNA problem for a given input file.

    Args:
        input_file_path (str): Path to input file

    Returns:
        int: Number of noncrossing matchings modulo 1,000,000
    """
    try:
        # Parse input
        rna = parse_input_file(input_file_path)
        print(f"Parsed input: RNA sequence = {rna[:10]}... (length={len(rna)})")

        # Initialize Motzkin counter
        counter = MotzkinRNA()

        # Compute the number of noncrossing matchings
        result = counter.count_noncrossing_matchings(rna)

        # Verify for small cases
        is_valid, msg = counter.verify_count(rna, result)
        if not is_valid:
            raise ValueError(f"Invalid result: {msg}")
        print(f"Verification: {msg}")

        return result

    except Exception as e:
        raise ValueError(f"Error solving problem: {str(e)}")

def main():
    """
    Main function to run the Motzkin RNA problem solver.
    """
    # Configuration
    input_file = "rosalind_motz.txt"  # Change this to your input file name
    output_file = "output_motz.txt"

    try:
        print("Solving Motzkin RNA Secondary Structures Problem...")

        # Solve the problem
        result = solve_motzkin_rna_problem(input_file)

        # Display results
        print(f"\nResult:")
        print(f"Number of noncrossing matchings (mod 1,000,000): {result}")

        # Write to output file
        write_output_file(output_file, result)
        print(f"Result written to: {output_file}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
        print("Please make sure the file exists in the current directory.")

    except ValueError as e:
        print(f"Error: {e}")

    except Exception as e:
        print(f"Unexpected error: {e}")

def test_motzkin_rna():
    """Test the Motzkin RNA counter with sample and edge cases."""
    print("=== Testing Motzkin RNA Counter ===")

    test_cases = [
        {"rna": "AUAU", "expected": 7},
        {"rna": "AU", "expected": 2},
        {"rna": "A", "expected": 1},
        {"rna": "GC", "expected": 2},
        {"rna": "AA", "expected": 1},  # Can't pair
        {"rna": "AUAUAU", "expected": 22},
    ]

    counter = MotzkinRNA()

    for i, case in enumerate(test_cases, 1):
        rna = case["rna"]
        expected = case.get("expected", None)
        print(f"\nTest {i}: RNA={rna}")

        result = counter.count_noncrossing_matchings(rna)
        print(f"Result: {result}")

        is_valid, msg = counter.verify_count(rna, result)
        print(f"Valid: {'✓' if is_valid else '✗'} - {msg}")

        if expected is not None and result == expected:
            print(f"Matches expected {expected}: ✓")
        elif expected is not None:
            print(f"Matches expected {expected}: ✗")

if __name__ == "__main__":
    # Run tests
    test_motzkin_rna()

    print("\n" + "="*60)

    # Run main function
    print("Running main function...")
    main()


## Majority Element Problem

In [ ]:
"""
Rosalind Majority Element Problem Solution


This module implements a divide-and-conquer algorithm to find the majority element
in multiple arrays. It handles large inputs efficiently and includes verification for correctness.


Author: Bioinformatics Solution
Compatible with: Python 3.6+
Platform: Google Colab
"""


from typing import List, Tuple



class MajorityFinder:
    """
    A class to find the majority element in an array using divide-and-conquer.
    """


    def __init__(self):
        """Initialize the majority finder."""
        pass


    def find_majority(self, arr: List[int]) -> int:
        """
        Find the majority element in the array using divide-and-conquer.


        A majority element appears more than n/2 times.


        Args:
            arr (List[int]): Input array


        Returns:
            int: The majority element if it exists, -1 otherwise
        """
        if not arr:
            return -1


        def majority_rec(start: int, end: int) -> int:
            if start == end:
                return arr[start]


            mid = (start + end) // 2
            left_major = majority_rec(start, mid)
            right_major = majority_rec(mid + 1, end)


            if left_major == right_major:
                return left_major


            # Count occurrences in the range
            left_count = sum(1 for i in range(start, end + 1) if arr[i] == left_major)
            right_count = sum(1 for i in range(start, end + 1) if arr[i] == right_major)


            if left_count > (end - start + 1) // 2:
                return left_major
            if right_count > (end - start + 1) // 2:
                return right_major


            return -1  # No majority in this range


        candidate = majority_rec(0, len(arr) - 1)
        if candidate == -1:
            return -1


        # Verify the count
        count = sum(1 for x in arr if x == candidate)
        return candidate if count > len(arr) // 2 else -1


    def verify_majority(self, arr: List[int], result: int) -> Tuple[bool, str]:
        """
        Verify that the majority element is correct.


        Args:
            arr (List[int]): Input array
            result (int): Reported majority element or -1


        Returns:
            Tuple[bool, str]: (is_valid, error_message)
        """
        if not arr:
            return result == -1, "Valid for empty array" if result == -1 else "Invalid for empty array"


        from collections import Counter
        counts = Counter(arr)
        n = len(arr)
        threshold = n // 2


        if result == -1:
            # Check if no element exceeds threshold
            if all(count <= threshold for count in counts.values()):
                return True, "Valid: No majority element"
            else:
                max_elem = max(counts, key=counts.get)
                return False, f"Invalid: {max_elem} appears {counts[max_elem]} > {threshold} times"
        else:
            # Check if result appears > n/2 times
            if result not in counts:
                return False, f"Result {result} not in array"
            if counts[result] <= threshold:
                return False, f"Result {result} appears {counts[result]} <= {threshold} times"


            # Check if it's indeed the majority
            if counts[result] > threshold:
                return True, f"Valid: {result} appears {counts[result]} > {threshold} times"
            return False, "Invalid majority"


def parse_input_file(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Parse input file to extract k, n, and k arrays each of size n.
    """
    try:
        with open(file_path, 'r') as file:
            lines = [line.strip() for line in file if line.strip()]


        if len(lines) < 1:
            raise ValueError("Input file is empty")


        # Parse k and n
        first_line = list(map(int, lines[0].split()))
        if len(first_line) != 2:
            raise ValueError("First line must contain exactly two integers: k and n")
        k, n = first_line


        if len(lines) != k + 1:
            raise ValueError(f"Expected {k + 1} lines, found {len(lines)}")


        # Parse k arrays
        arrays = []
        for i in range(1, k + 1):
            arr = list(map(int, lines[i].split()))
            if len(arr) != n:
                raise ValueError(f"Array {i} length {len(arr)} doesn't match n={n}")
            arrays.append(arr)


        return k, n, arrays


    except FileNotFoundError:
        raise FileNotFoundError(f"Input file '{file_path}' not found")
    except ValueError as e:
        raise ValueError(f"Input parsing error: {e}")



def write_output_file(output_path: str, results: List[int]) -> None:
    """
    Write majority elements to output file.
    """
    try:
        with open(output_path, 'w') as file:
            file.write(' '.join(map(str, results)) + '\n')
    except Exception as e:
        raise IOError(f"Error writing to output file: {e}")



def solve_majority_problem(input_file_path: str) -> List[int]:
    """
    Solve the majority element problem for a given input file.
    """
    try:
        # Parse input
        k, n, arrays = parse_input_file(input_file_path)


        print(f"Parsed input: k={k}, n={n}, {k} arrays")


        # Initialize finder
        finder = MajorityFinder()


        # Find majority for each array
        results = []
        for i, arr in enumerate(arrays, 1):
            major = finder.find_majority(arr)
            results.append(major)


            # Verify
            is_valid, msg = finder.verify_majority(arr, major)
            if not is_valid:
                raise ValueError(f"Invalid result for array {i}: {msg}")


            print(f"Array {i}: majority {major} - {msg}")


        return results


    except Exception as e:
        raise ValueError(f"Error solving problem: {str(e)}")



def main():
    """
    Main function to run the majority element problem solver.
    """
    # Configuration
    input_file = "rosalind_maj.txt"  # Change this to your input file name
    output_file = "output_maj.txt"


    try:
        print("Solving Majority Element Problem...")


        # Solve the problem
        results = solve_majority_problem(input_file)


        # Display results
        print(f"\nResult:")
        print(f"Majority elements: {' '.join(map(str, results))}")


        # Write to output file
        write_output_file(output_file, results)
        print(f"Result written to: {output_file}")


    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
        print("Please make sure the file exists in the current directory.")


    except ValueError as e:
        print(f"Error: {e}")


    except Exception as e:
        print(f"Unexpected error: {e}")



def test_majority_finder():
    """Test the majority finder with sample and edge cases."""
    print("=== Testing Majority Finder ===")


    test_cases = [
        # Sample dataset arrays
        [5, 5, 5, 5, 5, 5, 5, 5],                # 5 is majority
        [8, 7, 7, 7, 1, 7, 3, 7],                # 7 is majority (5 times > 4)
        [7, 1, 6, 5, 10, 100, 1000, 1],          # No majority
        [5, 1, 6, 7, 1, 1, 10, 1],               # 1 appears 4 times == 4, but need >4? Wait, n=8, >4
        # Note: for n=8, >4 means at least 5
        # In sample, last one has 1 appearing 4 times, so -1
        # Additional tests
        [],                                      # Empty
        [1],                                     # Single element
        [1, 2],                                  # No majority
        [1, 1, 2],                               # 1 appears 2 > 1.5
        [1, 2, 2, 3],                            # No majority
        [2, 2, 2, 2],                            # 2 is majority
    ]


    expected = [5, 7, -1, -1, -1, 1, -1, 1, -1, 2]


    finder = MajorityFinder()


    for i, arr in enumerate(test_cases, 1):
        print(f"\nTest {i}: {arr}")


        result = finder.find_majority(arr)
        print(f"Result: {result}")


        is_valid, msg = finder.verify_majority(arr, result)
        print(f"Valid: {'✓' if is_valid else '✗'} - {msg}")


        if i <= len(expected):
            exp = expected[i-1]
            print(f"Matches expected {exp}: {'✓' if result == exp else '✗'}")


if __name__ == "__main__":
    # Run tests
    test_majority_finder()


    print("\n" + "="*60)


    # Run main function
    print("Running main function...")
    main()
